In [1]:
from datetime import date, datetime, timedelta
import json
from time import localtime, sleep, strftime, time
import os

from dotenv import load_dotenv
from openpyxl import load_workbook
import pandas as pd
from seleniumwire import webdriver
from seleniumwire.utils import decode
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from timeit import default_timer
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
fb_mkp_ropa = {
    "Fecha Extraccion": [],
    "titulo_marketplace":[],
    "tiempo_creacion": [],
    "tipo_delivery": [],
    "delivery_data":[],
    "delivery_direccion":[],
    "descripcion": [],
    "disponible": [],
    "vendido": [],
    "fecha_union_vendedor":[],
    "cantidad": [],
    "precio": [],
    "tipo_moneda": [],
    "amount_with_concurrency":[],
    "latitud": [],
    "longitud": [],
    "locacion":[],
    "locacion_id":[],
    "name_vendedor":[],
    "tipo_vendedor":[],
    "id_vendedor":[]
}

In [3]:
fb_mkp_ropa_time = {
    "Fecha" : None,
    "Hora Inicio": None,
    "Hora Termino": None,
    "Cantidad": None,
    "Tiempo(HHMMSS)": None,
    "Productos/min": None,
    "Enlace": None,
    "Observaciones": None,
}

In [4]:
class ScraperFb:
    """Representa a un bot para hacer web scarping en fb marketplace.

    Attributes:
        driver (Object): Maneja un navegador para hacer web scraping
        wait (Object): Maneja el Tiempo de espera durante la ejecución del bot
    """
    
    def __init__(self):
        """Inicializa un objeto de tipo ScraperFb.

        Args:
            driver (Object): [Driver]
            wait (Object): [Wait]
        """
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(chrome_options=chrome_options,service=Service(ChromeDriverManager().install()))
        self.wait = WebDriverWait(self.driver, 10)

    def iniciar_sesion(self, url):
        """Inicia sesión en una página web usando un usuario y contraseña

        Args:
            url (str): [Url]
        """
        self.driver.get(url)
        self.driver.maximize_window()
        username = self.wait.until(EC.presence_of_element_located((By.ID, "email")))
        password = self.wait.until(EC.presence_of_element_located((By.ID, "pass")))
        username.clear()
        password.clear()
        username.send_keys(os.getenv('FB_USERNAME'))
        password.send_keys(os.getenv('FB_PASSWORD'))
        self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='login']"))).click()
    
    def mapear_datos(self):
        sleep(10)
        self.driver.execute_script("window.open('about:blank', 'newtab');")
        self.driver.switch_to.window("newtab")
        self.driver.get("https://www.facebook.com/marketplace/category/apparel/?sortBy=creation_time_descend&exact=false")
        sleep(8)        
        ropa = self.driver.find_elements(By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]')
        tiempo = fb_mkp_ropa_time["Fecha"]
        i=0
        e=0
        while tiempo >= '2023-01-05':
            print("Scrapeando item", i + 1)
            try:
                ropa[i].click()
                sleep(3)
                for request in self.driver.requests:
                    if not request.response or 'graphql' not in request.url:
                        continue
                    
                    body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
                    decoded_body = body.decode('utf-8')
                    json_data = json.loads(decoded_body)
                    
                    if 'prefetch_uris_v2' not in json_data['extensions']:
                        continue

                    print(localtime(json_data['data']['viewer']['marketplace_product_details_page']['target']['creation_time']))
                    tiempo = strftime('%Y-%m-%d', localtime(json_data['data']['viewer']['marketplace_product_details_page']['target']['creation_time']))
                    
                    if tiempo == '2023-01-05':
                        dato = json_data['data']['viewer']['marketplace_product_details_page']
                        print(dato["target"]["marketplace_listing_title"])
                        self.extraer_datos(dato)
                    break
                self.driver.execute_script("window.history.go(-1)");
                
            except (NoSuchElementException, StaleElementReferenceException) as error:
                print("Error:",error)
                print('No se hallo el item N '+str(i + 1)+'se pasará al siguiente')
                e=e+1
                
            except (KeyError, ElementNotInteractableException) as error:
                print("Error:",error)
                print('No se puede obtener la data del item N '+str(i + 1)+'se pasará al siguiente')
                e=e+1
                self.driver.execute_script("window.history.go(-1)");
                
            except Exception as error:
                print("Error:", error)
                e=e+1
                with open('data.json', 'w') as fp:
                    json.dump(fb_mkp_ropa, fp)
                exit()
            i = i + 1
            if i == len(ropa):
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                sleep(7)
                ropa = self.driver.find_elements(By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]')
            del self.driver.requests
            sleep(3)
            print('-------------------------------------------------------------------')
        fb_mkp_ropa_time["Cantidad"]= i - e
        print('-------------------------------------------------------------------')
        print('Fin de la extraccion')
        print('-------------------------------------------------------------------')
    
    def extraer_datos(self, item):
        fb_mkp_ropa["titulo_marketplace"].append(item['target']['marketplace_listing_title'])
        fb_mkp_ropa["tiempo_creacion"].append(item['target']['creation_time'])
        fb_mkp_ropa["tipo_delivery"].append(item['target']['delivery_types'][0])
        fb_mkp_ropa["delivery_data"].append(item['target']["delivery_data"].get('carrier', 'null'))
        fb_mkp_ropa["delivery_direccion"].append(item['target']["delivery_data"].get('delivery_address', 'null'))
        fb_mkp_ropa["descripcion"].append(item['target']['redacted_description'].get('text', 'null'))
        fb_mkp_ropa["disponible"].append(item['target']['is_live'])
        fb_mkp_ropa["vendido"].append(item['target']['is_sold'])
        fb_mkp_ropa["fecha_union_vendedor"].append(item['target']['marketplace_listing_seller'].get('join_time', 'null'))
        fb_mkp_ropa["cantidad"].append(item['target']['listing_inventory_type'])
        fb_mkp_ropa["precio"].append(item['target']['listing_price'].get('amount', 'null'))
        fb_mkp_ropa["tipo_moneda"].append(item['target']['listing_price'].get('currency', 'null'))
        fb_mkp_ropa["amount_with_concurrency"].append(item['target']['listing_price'].get('amount_with_offset_in_currency', 'null'))
        fb_mkp_ropa["latitud"].append(item['target']['location'].get('latitude', 'null'))
        fb_mkp_ropa["longitud"].append(item['target']['location'].get('longitude', 'null'))
        fb_mkp_ropa["locacion_id"].append(item['target']['location_vanity_or_id'])
        fb_mkp_ropa["name_vendedor"].append(item['target']['story']['actors'][0]['name'])
        fb_mkp_ropa["tipo_vendedor"].append(item['target']['story']['actors'][0]['__typename'])
        fb_mkp_ropa["id_vendedor"].append(item['target']['story']['actors'][0]['id'])
        #fb_mkp_ropa["Fecha Extraccion"].append(datetime.strptime(fb_mkp_ropa_time["Fecha"],"%Y-%m-%d").strftime('%d/%m/%Y'))
        fb_mkp_ropa["Fecha Extraccion"].append("04/01/2023")
        try:
            fb_mkp_ropa["locacion"].append(item['target']['location_text'].get('text', 'null'))
        except:
            fb_mkp_ropa["locacion"].append("null")

    def guardar_datos(self):
        df_fb_mkp_ropa = pd.DataFrame(fb_mkp_ropa)
        df_fb_mkp_ropa.drop(len(df_fb_mkp_ropa)-1, axis=0, inplace=True)
        df_fb_mkp_ropa.fillna("null")
        datetime_obj = datetime.strptime(fb_mkp_ropa_time["Fecha"],"%Y-%m-%d")
        path = "Data/" + datetime_obj.strftime('%d-%m-%Y') + "/"
        filename = "fb_ropa_" + datetime_obj.strftime('%d%m%Y') + "_" + str(fb_mkp_ropa_time["Cantidad"]) + ".xlsx"
        if not os.path.exists(path):
            os.mkdir(path)
        df_fb_mkp_ropa.to_excel(path + filename)
        
    def guardar_tiempos(self):
        filename = 'Tiempos.xlsx'
        tiempos = load_workbook(filename)
        sheet_name = "Ropa"
        header_exist = True
        if sheet_name not in [ws.title for ws in tiempos.worksheets]:
            tiempos.create_sheet(sheet_name)
            header_exist = False
        worksheet = tiempos[sheet_name]
        if not header_exist:
            worksheet.append(list(fb_mkp_ropa_time.keys()))
        worksheet.append(list(fb_mkp_ropa_time.values()))
        tiempos.save(filename)
        tiempos.close()

In [5]:
def set_params_inicio():
    fb_mkp_ropa_time["Fecha"] = str(datetime.now().date())
    start = time()
    fb_mkp_ropa_time["Hora Inicio"] = strftime("%H:%M:%S", localtime(start))
    print("Hora de inicio:",fb_mkp_ropa_time["Hora Inicio"])
    return start

def set_params_final(start):
    end = time()
    fb_mkp_ropa_time["Hora Termino"] = strftime("%H:%M:%S", localtime(end))
    print("Hora Termino:",fb_mkp_ropa_time["Hora Termino"])
    total = end - start
    print("Duracion: ",total, 'seconds')
    fb_mkp_ropa_time["Tiempo(HHMMSS)"] = str(timedelta(seconds=total)).split(".")[0]
    fb_mkp_ropa_time["Productos/min"] = int(fb_mkp_ropa_time["Cantidad"]/(total / 60))

In [6]:
def main():
    load_dotenv()
    start = set_params_inicio()
    url = 'https://www.facebook.com/'
    scraper = ScraperFb()
    scraper.iniciar_sesion(url)
    scraper.mapear_datos()
    scraper.guardar_datos()
    set_params_final(start)
    scraper.guardar_tiempos()

In [7]:
if __name__ == '__main__':
    main()

Hora de inicio: 10:51:01
Scrapeando item 1
Error: 'marketplace_product_details_page'
No se puede obtener la data del item N 1se pasará al siguiente
-------------------------------------------------------------------
Scrapeando item 2
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=38, tm_sec=33, tm_wday=4, tm_yday=6, tm_isdst=0)
Pollo algodón rip fresco
-------------------------------------------------------------------
Scrapeando item 3
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=37, tm_sec=30, tm_wday=4, tm_yday=6, tm_isdst=0)
Polos camiseros con logo bordado
-------------------------------------------------------------------
Scrapeando item 4
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=34, tm_sec=57, tm_wday=4, tm_yday=6, tm_isdst=0)
Zapatos
-------------------------------------------------------------------
Scrapeando item 5
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=31, tm_sec=

time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=4, tm_sec=10, tm_wday=4, tm_yday=6, tm_isdst=0)
Polera Bearcliff hombre
-------------------------------------------------------------------
Scrapeando item 37
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=4, tm_sec=2, tm_wday=4, tm_yday=6, tm_isdst=0)
Nike Air Forcé One Shadow 
Talla 37.5(23.5)
-------------------------------------------------------------------
Scrapeando item 38
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=3, tm_sec=41, tm_wday=4, tm_yday=6, tm_isdst=0)
Tops talla S 💟
-------------------------------------------------------------------
Scrapeando item 39
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, tm_min=1, tm_sec=51, tm_wday=4, tm_yday=6, tm_isdst=0)
Casaca Tommy Hilfiger hombre
-------------------------------------------------------------------
Scrapeando item 40
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=10, t

time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=43, tm_sec=33, tm_wday=4, tm_yday=6, tm_isdst=0)
Zapatillas VANS
-------------------------------------------------------------------
Scrapeando item 72
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=43, tm_sec=31, tm_wday=4, tm_yday=6, tm_isdst=0)
Vestido Gasa Manga Larga
-------------------------------------------------------------------
Scrapeando item 73
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=42, tm_sec=20, tm_wday=4, tm_yday=6, tm_isdst=0)
Camiseta original
-------------------------------------------------------------------
Scrapeando item 74
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=42, tm_sec=15, tm_wday=4, tm_yday=6, tm_isdst=0)
Cortaviento tommy vintage
-------------------------------------------------------------------
Scrapeando item 75
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=41, tm_sec=24, tm_wda

-------------------------------------------------------------------
Scrapeando item 107
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=22, tm_sec=13, tm_wday=4, tm_yday=6, tm_isdst=0)
Airmax Command(40.5)
-------------------------------------------------------------------
Scrapeando item 108
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=21, tm_sec=37, tm_wday=4, tm_yday=6, tm_isdst=0)
Jordan talla 31
-------------------------------------------------------------------
Scrapeando item 109
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=20, tm_sec=9, tm_wday=4, tm_yday=6, tm_isdst=0)
H0003 - Zapatillas Nike Adapt BB Mag (US Charger)
-------------------------------------------------------------------
Scrapeando item 110
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=19, tm_sec=58, tm_wday=4, tm_yday=6, tm_isdst=0)
Leggin importadas tie dye
------------------------------------------------------------

-------------------------------------------------------------------
Scrapeando item 144
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=30, tm_sec=0, tm_wday=4, tm_yday=6, tm_isdst=0)
Shorts y tops
-------------------------------------------------------------------
Scrapeando item 145
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=10, tm_sec=41, tm_wday=4, tm_yday=6, tm_isdst=0)
Botín Combat y zapatilla 🍀
-------------------------------------------------------------------
Scrapeando item 146
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=7, tm_sec=41, tm_wday=4, tm_yday=6, tm_isdst=0)
Camisa
-------------------------------------------------------------------
Scrapeando item 147
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=6, tm_hour=9, tm_min=4, tm_sec=45, tm_wday=4, tm_yday=6, tm_isdst=0)
Zapatillas Nike sb Rodriguez talla 9us
-------------------------------------------------------------------
Scrapeando item 148


...

-------------------------------------------------------------------
Scrapeando item 413
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=16, tm_min=0, tm_sec=1, tm_wday=3, tm_yday=5, tm_isdst=0)
Liquidacion Nike Air Force One Blancas
-------------------------------------------------------------------
Scrapeando item 414
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=15, tm_min=58, tm_sec=5, tm_wday=3, tm_yday=5, tm_isdst=0)
Zapatillas guess
-------------------------------------------------------------------
Scrapeando item 415
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=15, tm_min=53, tm_sec=57, tm_wday=3, tm_yday=5, tm_isdst=0)
Converse CT AS Glitters
-------------------------------------------------------------------
Scrapeando item 416
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=15, tm_min=45, tm_sec=2, tm_wday=3, tm_yday=5, tm_isdst=0)
Short OAKLEY talla 36 Estado 9/10
----------------------------------------------------------

-------------------------------------------------------------------
Scrapeando item 448
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=13, tm_min=6, tm_sec=57, tm_wday=3, tm_yday=5, tm_isdst=0)
Nike Air Force 1
-------------------------------------------------------------------
Scrapeando item 449
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=13, tm_min=1, tm_sec=52, tm_wday=3, tm_yday=5, tm_isdst=0)
Chor rigido talla 30 nuevo original de hombre
-------------------------------------------------------------------
Scrapeando item 450
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=13, tm_min=1, tm_sec=31, tm_wday=3, tm_yday=5, tm_isdst=0)
Closet sale 💅🏼
-------------------------------------------------------------------
Scrapeando item 451
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=13, tm_min=0, tm_sec=14, tm_wday=3, tm_yday=5, tm_isdst=0)
Zapatillas Nike
-------------------------------------------------------------------
Scrapeando

-------------------------------------------------------------------
Scrapeando item 483
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=11, tm_min=40, tm_sec=26, tm_wday=3, tm_yday=5, tm_isdst=0)
Mayze Originales En Oferta Talla 38
-------------------------------------------------------------------
Scrapeando item 484
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=11, tm_min=40, tm_sec=7, tm_wday=3, tm_yday=5, tm_isdst=0)
ZAPATILLAS REEBOK HOMBRE ZIG DYNAMICA 2.0❗️
-------------------------------------------------------------------
Scrapeando item 485
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=11, tm_min=26, tm_sec=8, tm_wday=3, tm_yday=5, tm_isdst=0)
Zapatillas Nike original tal cual la foto 🤓 talla 40 no 🐸
-------------------------------------------------------------------
Scrapeando item 486
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=11, tm_min=21, tm_sec=11, tm_wday=3, tm_yday=5, tm_isdst=0)
Zapatillas en muy buen estado ta

time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=0, tm_min=29, tm_sec=14, tm_wday=3, tm_yday=5, tm_isdst=0)
NIKE AIR MAX TERRASCAPE 97
-------------------------------------------------------------------
Scrapeando item 518
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=0, tm_min=24, tm_sec=10, tm_wday=3, tm_yday=5, tm_isdst=0)
Ropas
-------------------------------------------------------------------
Scrapeando item 519
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=0, tm_min=17, tm_sec=49, tm_wday=3, tm_yday=5, tm_isdst=0)
POLOS oversize unisex..s..m..L..xl
-------------------------------------------------------------------
Scrapeando item 520
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, tm_hour=0, tm_min=17, tm_sec=29, tm_wday=3, tm_yday=5, tm_isdst=0)
Closet sale. Vestido brilloso, espalda descubierta✨
-------------------------------------------------------------------
Scrapeando item 521
time.struct_time(tm_year=2023, tm_mon=1, tm_mday=5, 

ValueError: All arrays must be of the same length